<a href="https://colab.research.google.com/github/bolulka/NeuralNetworks/blob/main/lab9/lab9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Лаб. 9.1
Задание 1.  (из  темы  9.1):

Дообучите нейронную сеть VGG16 для распознавания изображений только двух классов. Используйте дообученную сеть для классификации своих изображений.

Примеры для темы 9

Задание 2.  (из  темы  9.2):

Дообучите нейронную сеть ResNet34 распознаванию Вашего лица на фотографии. Используйте дообученную сеть для распознавания Вашего лица на других фотографиях (другой возраст; другой ракурс; часть лица закрыта, например, очками).

Пример кода: https://github.com/sozykin/dlpython_course/blob/master/computer_vision/foto_comparison/foto_verification.ipynb

В качестве входных данных подайте свои изображения (задания 1-2).

В качестве результатов - результаты распознавания (задание 1) и анализ результатов (задание 2).

In [ ]:
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model

# Размер изображений
img_width, img_height = 150, 150
# Путь к каталогу с изображениями для обучения
train_data_dir = '/content/drive/MyDrive/Colab Notebooks/data/train'
# Путь к каталогу с изображениями для валидации
validation_data_dir = '/content/drive/MyDrive/Colab Notebooks/data/validation'
# Количество изображений для обучения
nb_train_samples = 32
# Количество изображений для валидации
nb_validation_samples = 8
# Количество эпох
epochs = 5
# Размер выборки
batch_size = 16

# Загружаем сеть VGG16 без части, которая отвечает за классификацию
base_model = applications.VGG16(weights='imagenet', include_top=False)

# Добавляем слои классификации
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
# Выходной слой с двумя нейронами для классов "кот" и "собака"
predictions = Dense(2, activation='softmax')(x)

# Составляем сеть из двух частей
model = Model(inputs=base_model.input, outputs=predictions)

# "Замораживаем" сверточные уровни сети VGG16
# Обучаем только вновь добавленные слои
for layer in base_model.layers:
    layer.trainable = False

# Компилируем модель
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Создаем генератор данных для обучения
datagen = ImageDataGenerator(rescale=1. / 255)
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Создаем генератор данных для валидации
validation_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Обучаем модель с помощью генератора
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples)

print("Сохраняем сеть")
# Сохраняем сеть для последующего использования
# Генерируем описание модели в формате json
model_json = model.to_json()
json_file = open("vgg16_cat_dogs.json", "w")
# Записываем архитектуру сети в файл
json_file.write(model_json)
json_file.close()
# Записываем данные о весах в файл
model.save_weights("vgg16_cat_dogs.h5")
print("Сохранение сети завершено")


In [ ]:
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import matplotlib.pyplot as plt

# Список классов
classes = ['кот', 'собака']

#Загружаем обученную модель
json_file = open("vgg16_cat_dogs.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("vgg16_cat_dogs.h5")

#Компилируем модель
loaded_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

#Загружаем изображение для распознавания
img = image.load_img('dog.jpg', target_size=(224, 224))
# img = image.load_img('cat_dog.jpg', target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

#Запускаем распознавание
prediction = loaded_model.predict(x)
print(prediction)
print(classes[np.argmax(prediction)])

plt.imshow(img)
plt.show()


In [ ]:
import dlib
from skimage import io
from scipy.spatial import distance

sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
facerec = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')
detector = dlib.get_frontal_face_detector()

img = io.imread('pic1.jpg')

dets = detector(img, 1)
for k, d in enumerate(dets):
    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
        k, d.left(), d.top(), d.right(), d.bottom()))
    shape = sp(img, d)


face_descriptor1 = facerec.compute_face_descriptor(img, shape)
# print(face_descriptor1)

img = io.imread('pic2.jpg')
dets_webcam = detector(img, 1)
for k, d in enumerate(dets_webcam):
    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
        k, d.left(), d.top(), d.right(), d.bottom()))
    shape = sp(img, d)

face_descriptor2 = facerec.compute_face_descriptor(img, shape)

a = distance.euclidean(face_descriptor1, face_descriptor2)
print(a)

